# Human Activity Recognition - Prediction Script

This notebook loads the trained LSTM model and predicts activities from new sensor data.

# Imports and Setup

In [ ]:
import numpy as np
import pandas as pd
import json
from scipy import signal

# TensorFlow 1.x compatibility
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

# ✅ CRITICAL: Clear any previous graph
tf.reset_default_graph()

print("✓ TensorFlow loaded in compatibility mode")

Instructions for updating:
non-resource variables are not supported in the long term
✓ TensorFlow loaded in compatibility mode


/Users/didiermupenda/miniforge3/envs/dl-projects/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


# Load Model Configuration

In [ ]:
# Load model metadata saved during training
with open('./model/model_info.json', 'r') as f:
    model_info = json.load(f)

n_hidden = model_info['n_hidden']
n_classes = model_info['n_classes']
n_steps = model_info['n_steps']
n_input = model_info['n_input']

LABELS = [
    "WALKING",
    "WALKING_UPSTAIRS",
    "WALKING_DOWNSTAIRS",
    "SITTING",
    "STANDING",
    "LAYING"
]

print(f"Model Configuration:")
print(f"  - Input: {n_steps} timesteps × {n_input} features")
print(f"  - Hidden: {n_hidden} units")
print(f"  - Classes: {n_classes}")
print(f"  - Training accuracy: {model_info['final_accuracy']*100:.2f}%")

Model Configuration:
  - Input: 128 timesteps × 9 features
  - Hidden: 32 units
  - Classes: 6
  - Training accuracy: 88.39%


# Rebuild Model Architecture (MUST match training exactly!)

In [ ]:
def LSTM_RNN(_X, _weights, _biases):
    """
    LSTM neural network - must be IDENTICAL to training code
    """
    _X = tf.transpose(_X, [1, 0, 2])  # permute n_steps and batch_size
    _X = tf.reshape(_X, [-1, n_input])
    
    # ReLU activation
    _X = tf.nn.relu(tf.matmul(_X, _weights['hidden']) + _biases['hidden'])
    _X = tf.split(_X, n_steps, 0)
    
    # Two stacked LSTM cells
    from tensorflow.python.ops import rnn_cell_impl
    lstm_cell_1 = rnn_cell_impl.LSTMCell(n_hidden, forget_bias=1.0)
    lstm_cell_2 = rnn_cell_impl.LSTMCell(n_hidden, forget_bias=1.0)
    lstm_cells = rnn_cell_impl.MultiRNNCell([lstm_cell_1, lstm_cell_2])
    
    outputs, states = tf.nn.static_rnn(lstm_cells, _X, dtype=tf.float32)
    lstm_last_output = outputs[-1]
    
    return tf.matmul(lstm_last_output, _weights['out']) + _biases['out']

# Create placeholders and variables - NAMES MUST MATCH TRAINING!
x = tf.placeholder(tf.float32, [None, n_steps, n_input], name='input_x')

weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden]), name='weights_hidden'),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=1.0), name='weights_out')
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden]), name='biases_hidden'),
    'out': tf.Variable(tf.random_normal([n_classes]), name='biases_out')
}

pred = LSTM_RNN(x, weights, biases)

print("✓ Model architecture created")

Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


/Users/didiermupenda/miniforge3/envs/dl-projects/lib/python3.12/site-packages/tensorflow/python/keras/layers/legacy_rnn/rnn_cell_impl.py:910: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  warnings.warn("`tf.nn.rnn_cell.LSTMCell` is deprecated and will be "
/Users/didiermupenda/miniforge3/envs/dl-projects/lib/python3.12/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1702: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


✓ Model architecture created


# Sensor Data Preprocessing Function

In [ ]:
def preprocess_sensor_data(csv_file, window_size=128, overlap=0.5, input_hz=100, target_hz=50):
    """
    Preprocess raw sensor data from CSV into model-ready format.
    
    The UCI HAR dataset uses 9 features per timestep at 50 Hz:
    - body_acc_x, body_acc_y, body_acc_z (body acceleration, gravity removed)
    - body_gyro_x, body_gyro_y, body_gyro_z (angular velocity)
    - total_acc_x, total_acc_y, total_acc_z (raw accelerometer)
    
    IMPORTANT: Phone orientation must match UCI HAR - X-axis pointing UP when standing
    
    Args:
        csv_file: Path to CSV with columns: acc_x, acc_y, acc_z, gyro_x, gyro_y, gyro_z
        window_size: Number of timesteps per window (default 128, same as UCI HAR)
        overlap: Overlap between windows (default 0.5 = 50%)
        input_hz: Sampling rate of your sensor data (default 100 Hz for Sensor Logger at 10ms)
        target_hz: Target sampling rate for the model (default 50 Hz for UCI HAR)
    
    Returns:
        windows: Array of shape (num_windows, 128, 9)
    """
    # Read CSV - handle both comma and semicolon delimiters
    try:
        df = pd.read_csv(csv_file, delimiter=',')
        if len(df.columns) == 1:
            df = pd.read_csv(csv_file, delimiter=';')
    except:
        df = pd.read_csv(csv_file, delimiter=';')
    
    print(f"Loaded {csv_file}")
    print(f"  Columns: {list(df.columns)}")
    print(f"  Original shape: {df.shape}")
    
    # Expected columns
    required_cols = ['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z']
    
    # Check for missing columns
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}. Found: {list(df.columns)}")
    
    # Handle missing values
    df_clean = df[required_cols].dropna()
    
    # ============================================
    # DOWNSAMPLE from input_hz to target_hz (e.g., 100 Hz -> 50 Hz)
    # ============================================
    if input_hz != target_hz:
        downsample_factor = input_hz // target_hz
        print(f"  ⚡ Downsampling: {input_hz} Hz -> {target_hz} Hz (taking every {downsample_factor}th sample)")
        df_clean = df_clean.iloc[::downsample_factor].reset_index(drop=True)
        print(f"  Shape after downsampling: {df_clean.shape}")
    
    if len(df_clean) < window_size:
        raise ValueError(f"Need at least {window_size} samples, got {len(df_clean)}")
    
    # Extract sensor data
    acc = df_clean[['acc_x', 'acc_y', 'acc_z']].values
    gyro = df_clean[['gyro_x', 'gyro_y', 'gyro_z']].values

    # Apply Butterworth high-pass filter to separate body acceleration from gravity
    # UCI HAR uses 0.3 Hz cutoff at 50 Hz sampling rate
    b, a = signal.butter(3, 0.3, btype='high', fs=target_hz)
    body_acc = np.zeros_like(acc)
    for i in range(3):
        body_acc[:, i] = signal.filtfilt(b, a, acc[:, i])
    
    # Total acceleration is the raw accelerometer data
    total_acc = acc
    
    # Combine all 9 features in the same order as UCI HAR:
    # [body_acc_x, body_acc_y, body_acc_z, gyro_x, gyro_y, gyro_z, total_acc_x, total_acc_y, total_acc_z]
    all_features = np.hstack([body_acc, gyro, total_acc])
    
    # Create sliding windows
    step_size = int(window_size * (1 - overlap))
    windows = []
    
    for start in range(0, len(all_features) - window_size + 1, step_size):
        window = all_features[start:start + window_size]
        windows.append(window)
    
    windows = np.array(windows, dtype=np.float32)
    print(f"  Created {len(windows)} windows of shape (128, 9)")
    print(f"  Each window = {window_size / target_hz:.2f} seconds of activity")
    
    return windows

print("✓ Preprocessing function defined (100Hz → 50Hz downsampling)")

✓ Preprocessing function defined (with axis remapping & downsampling)


# Prediction Function

In [ ]:
def predict_activity(csv_file):
    """
    Load model, preprocess data, and predict activity.
    
    Args:
        csv_file: Path to CSV file with sensor data
    
    Returns:
        activity: Predicted activity label
        confidence: Confidence percentage
    """
    # Preprocess the sensor data
    windows = preprocess_sensor_data(csv_file)
    
    # Create saver and restore model
    saver = tf.train.Saver()
    
    with tf.Session() as sess:
        # Restore trained weights
        saver.restore(sess, "./model/lstm_model.ckpt")
        print("✓ Model restored from checkpoint")
        
        # Make predictions for all windows
        predictions_raw = sess.run(pred, feed_dict={x: windows})
        probabilities = sess.run(tf.nn.softmax(predictions_raw))
        predicted_classes = predictions_raw.argmax(axis=1)
        
        # Count predictions
        from collections import Counter
        vote_counts = Counter(predicted_classes)
        
        # Get the most common prediction
        most_common_class = vote_counts.most_common(1)[0][0]
        vote_count = vote_counts.most_common(1)[0][1]
        
        # Calculate average confidence for the predicted class
        confidences = [probabilities[i][predicted_classes[i]] 
                      for i in range(len(windows))]
        avg_confidence = np.mean(confidences) * 100
        
        # Print results
        print(f"\n{'='*50}")
        print(f"PREDICTION RESULTS")
        print(f"{'='*50}")
        print(f"\n  Activity: {LABELS[most_common_class]}")
        print(f"  Confidence: {avg_confidence:.1f}%")
        print(f"  Windows: {vote_count}/{len(windows)} voted for this class")
        print(f"\n  Vote breakdown:")
        for cls, count in vote_counts.most_common():
            pct = count / len(windows) * 100
            print(f"    {LABELS[cls]:20s}: {count:3d} ({pct:.1f}%)")
        print(f"{'='*50}\n")
        
        return LABELS[most_common_class], avg_confidence

print("✓ Prediction function defined")

✓ Prediction function defined


# Run Predictions

Change the file path below to predict on different sensor data files.

In [39]:
# ============================================
# Cell 6: Run Prediction on Your Data
# ============================================

# # Test with walking data
# activity, confidence = predict_activity('sensor_data/walking_data2.csv')

# Uncomment to test other activities:
activity, confidence = predict_activity('sensor_data/walking2_data.csv')
activity, confidence = predict_activity('sensor_data/standing2_data.csv')

Loaded sensor_data/walking2_data.csv
  Columns: ['timestamp', 'acc_z', 'acc_y', 'acc_x', 'gyro_z', 'gyro_y', 'gyro_x']
  Original shape: (6963, 7)
  ⚡ Downsampling: 100 Hz -> 50 Hz (taking every 2th sample)
  Shape after downsampling: (3482, 6)
  Created 53 windows of shape (128, 9)
  Each window = 2.56 seconds of activity
INFO:tensorflow:Restoring parameters from ./model/lstm_model.ckpt
✓ Model restored from checkpoint

PREDICTION RESULTS

  Activity: WALKING_DOWNSTAIRS
  Confidence: 98.3%
  Windows: 52/53 voted for this class

  Vote breakdown:
    WALKING_DOWNSTAIRS  :  52 (98.1%)
    WALKING             :   1 (1.9%)

Loaded sensor_data/standing2_data.csv
  Columns: ['timestamp', 'acc_z', 'acc_y', 'acc_x', 'gyro_z', 'gyro_y', 'gyro_x']
  Original shape: (3349, 7)
  ⚡ Downsampling: 100 Hz -> 50 Hz (taking every 2th sample)
  Shape after downsampling: (1675, 6)
  Created 25 windows of shape (128, 9)
  Each window = 2.56 seconds of activity
INFO:tensorflow:Restoring parameters from ./m

# DEBUG: Test model with UCI HAR training data


In [15]:
# ============================================
# DEBUG: Test model with UCI HAR training data
# If it fails on training data too, the model checkpoint is bad
# ============================================

# Load actual UCI HAR data that the model was trained on
uci_path = "UCI HAR Dataset/"
X_test_signals = []
for sig in ["body_acc_x_", "body_acc_y_", "body_acc_z_", 
            "body_gyro_x_", "body_gyro_y_", "body_gyro_z_",
            "total_acc_x_", "total_acc_y_", "total_acc_z_"]:
    data = np.loadtxt(f"{uci_path}test/Inertial Signals/{sig}test.txt")
    X_test_signals.append(data)

X_test = np.transpose(np.array(X_test_signals), (1, 2, 0))
y_test = np.loadtxt(uci_path + "test/y_test.txt", dtype=int) - 1  # 0-indexed

print(f"Loaded UCI HAR test data: {X_test.shape}")
print(f"Testing model on ORIGINAL training data...\n")

# Test with the model
saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "./model/lstm_model.ckpt")
    
    # Test on 10 samples of each class
    for class_idx in range(6):
        indices = np.where(y_test == class_idx)[0][:5]
        correct = 0
        for idx in indices:
            sample = X_test[idx:idx+1]
            pred_raw = sess.run(pred, feed_dict={x: sample})
            pred_class = np.argmax(pred_raw)
            if pred_class == class_idx:
                correct += 1
        print(f"{LABELS[class_idx]:20s}: {correct}/5 correct")
    
    # Overall accuracy on first 100 samples
    pred_all = sess.run(pred, feed_dict={x: X_test[:100]})
    pred_classes = np.argmax(pred_all, axis=1)
    accuracy = np.mean(pred_classes == y_test[:100])
    print(f"\nOverall accuracy on 100 test samples: {accuracy*100:.1f}%")
    print(f"\n⚠️ If accuracy is ~16-20%, the saved model is corrupted!")
    print(f"   You need to RETRAIN the model with a fresh kernel.")

Loaded UCI HAR test data: (2947, 128, 9)
Testing model on ORIGINAL training data...

INFO:tensorflow:Restoring parameters from ./model/lstm_model.ckpt
WALKING             : 5/5 correct
WALKING_UPSTAIRS    : 5/5 correct
WALKING_DOWNSTAIRS  : 5/5 correct
SITTING             : 5/5 correct
STANDING            : 4/5 correct
LAYING              : 5/5 correct

Overall accuracy on 100 test samples: 88.0%

⚠️ If accuracy is ~16-20%, the saved model is corrupted!
   You need to RETRAIN the model with a fresh kernel.


In [11]:
# ============================================
# TEST: Predict ONLY on STANDING samples (label 5)
# ============================================

# Filter for only STANDING samples (label 5 in file = index 4 after -1)
standing_indices = np.where(y_test == 4)[0]  # 4 because we did -1 earlier (5-1=4)
X_standing = X_test[standing_indices]
y_standing = y_test[standing_indices]

print(f"Testing ONLY on STANDING samples")
print(f"Number of STANDING samples: {len(X_standing)}")
print(f"Shape: {X_standing.shape}")
print(f"Expected label (0-indexed): 4 = STANDING\n")

saver = tf.train.Saver()
with tf.Session() as sess:
    saver.restore(sess, "./model/lstm_model.ckpt")
    
    # Predict on first 20 standing samples
    n_samples = min(20, len(X_standing))
    pred_all = sess.run(pred, feed_dict={x: X_standing[:n_samples]})
    pred_classes = np.argmax(pred_all, axis=1)
    
    print(f"Predictions for {n_samples} STANDING samples:")
    print("-" * 40)
    for i in range(n_samples):
        predicted = LABELS[pred_classes[i]]
        correct = "✓" if pred_classes[i] == 4 else "✗"
        print(f"  Sample {i+1}: {predicted:20s} {correct}")
    
    # Summary
    correct_count = np.sum(pred_classes == 4)
    print("-" * 40)
    print(f"\nAccuracy on STANDING: {correct_count}/{n_samples} = {correct_count/n_samples*100:.1f}%")
    
    # Show prediction distribution
    from collections import Counter
    counts = Counter(pred_classes)
    print(f"\nPrediction distribution:")
    for cls, count in sorted(counts.items()):
        print(f"  {LABELS[cls]:20s}: {count} ({count/n_samples*100:.1f}%)")

Testing ONLY on STANDING samples
Number of STANDING samples: 532
Shape: (532, 128, 9)
Expected label (0-indexed): 4 = STANDING

INFO:tensorflow:Restoring parameters from ./model/lstm_model.ckpt
Predictions for 20 STANDING samples:
----------------------------------------
  Sample 1: WALKING              ✗
  Sample 2: STANDING             ✓
  Sample 3: STANDING             ✓
  Sample 4: STANDING             ✓
  Sample 5: STANDING             ✓
  Sample 6: STANDING             ✓
  Sample 7: STANDING             ✓
  Sample 8: STANDING             ✓
  Sample 9: STANDING             ✓
  Sample 10: STANDING             ✓
  Sample 11: STANDING             ✓
  Sample 12: STANDING             ✓
  Sample 13: STANDING             ✓
  Sample 14: STANDING             ✓
  Sample 15: STANDING             ✓
  Sample 16: STANDING             ✓
  Sample 17: STANDING             ✓
  Sample 18: STANDING             ✓
  Sample 19: STANDING             ✓
  Sample 20: STANDING             ✓
-----------------

In [38]:
# ============================================
# DIAGNOSTIC: Compare your sensor data vs UCI HAR data
# ============================================

print("=" * 60)
print("COMPARING YOUR DATA vs UCI HAR DATA")
print("=" * 60)

# Load your standing data
df_yours = pd.read_csv('sensor_data/standing2_data.csv', delimiter=';')
df_yours = df_yours.iloc[::2]  # Downsample to 50Hz

# Load UCI HAR standing data (one sample = 128 timesteps)
uci_standing_idx = np.where(y_test == 4)[0][0]  # First STANDING sample
uci_standing = X_test[uci_standing_idx]  # Shape: (128, 9)

print("\n📊 YOUR STANDING DATA (first 128 samples after downsampling):")
print("-" * 60)
your_acc_x = df_yours['acc_x'].values[:128]
your_acc_y = df_yours['acc_y'].values[:128]
your_acc_z = df_yours['acc_z'].values[:128]
your_gyro_x = df_yours['gyro_x'].values[:128]
your_gyro_y = df_yours['gyro_y'].values[:128]
your_gyro_z = df_yours['gyro_z'].values[:128]

print(f"  acc_x:  mean={your_acc_x.mean():+.4f}, std={your_acc_x.std():.4f}, range=[{your_acc_x.min():+.3f}, {your_acc_x.max():+.3f}]")
print(f"  acc_y:  mean={your_acc_y.mean():+.4f}, std={your_acc_y.std():.4f}, range=[{your_acc_y.min():+.3f}, {your_acc_y.max():+.3f}]")
print(f"  acc_z:  mean={your_acc_z.mean():+.4f}, std={your_acc_z.std():.4f}, range=[{your_acc_z.min():+.3f}, {your_acc_z.max():+.3f}]")
print(f"  gyro_x: mean={your_gyro_x.mean():+.4f}, std={your_gyro_x.std():.4f}")
print(f"  gyro_y: mean={your_gyro_y.mean():+.4f}, std={your_gyro_y.std():.4f}")
print(f"  gyro_z: mean={your_gyro_z.mean():+.4f}, std={your_gyro_z.std():.4f}")

# Detect gravity axis
gravity_axis = "X" if abs(your_acc_x.mean()) > 0.8 else ("Y" if abs(your_acc_y.mean()) > 0.8 else "Z")
print(f"\n  🎯 GRAVITY detected on: {gravity_axis}-axis (value ≈ {[your_acc_x.mean(), your_acc_y.mean(), your_acc_z.mean()][['X','Y','Z'].index(gravity_axis)]:.2f}g)")

print("\n📊 UCI HAR STANDING DATA (sample #{})".format(uci_standing_idx))
print("-" * 60)
# UCI order: body_acc_x, body_acc_y, body_acc_z, gyro_x, gyro_y, gyro_z, total_acc_x, total_acc_y, total_acc_z
uci_total_acc_x = uci_standing[:, 6]
uci_total_acc_y = uci_standing[:, 7]
uci_total_acc_z = uci_standing[:, 8]
uci_gyro_x = uci_standing[:, 3]
uci_gyro_y = uci_standing[:, 4]
uci_gyro_z = uci_standing[:, 5]

print(f"  total_acc_x: mean={uci_total_acc_x.mean():+.4f}, std={uci_total_acc_x.std():.4f}, range=[{uci_total_acc_x.min():+.3f}, {uci_total_acc_x.max():+.3f}]")
print(f"  total_acc_y: mean={uci_total_acc_y.mean():+.4f}, std={uci_total_acc_y.std():.4f}, range=[{uci_total_acc_y.min():+.3f}, {uci_total_acc_y.max():+.3f}]")
print(f"  total_acc_z: mean={uci_total_acc_z.mean():+.4f}, std={uci_total_acc_z.std():.4f}, range=[{uci_total_acc_z.min():+.3f}, {uci_total_acc_z.max():+.3f}]")
print(f"  gyro_x:      mean={uci_gyro_x.mean():+.4f}, std={uci_gyro_x.std():.4f}")
print(f"  gyro_y:      mean={uci_gyro_y.mean():+.4f}, std={uci_gyro_y.std():.4f}")
print(f"  gyro_z:      mean={uci_gyro_z.mean():+.4f}, std={uci_gyro_z.std():.4f}")

uci_gravity_axis = "X" if abs(uci_total_acc_x.mean()) > 0.8 else ("Y" if abs(uci_total_acc_y.mean()) > 0.8 else "Z")
print(f"\n  🎯 GRAVITY detected on: {uci_gravity_axis}-axis (value ≈ {[uci_total_acc_x.mean(), uci_total_acc_y.mean(), uci_total_acc_z.mean()][['X','Y','Z'].index(uci_gravity_axis)]:.2f}g)")

# Now check UCI LAYING data for comparison
print("\n📊 UCI HAR LAYING DATA (for comparison)")
print("-" * 60)
uci_laying_idx = np.where(y_test == 5)[0][0]  # First LAYING sample
uci_laying = X_test[uci_laying_idx]
uci_lay_acc_x = uci_laying[:, 6]
uci_lay_acc_y = uci_laying[:, 7]
uci_lay_acc_z = uci_laying[:, 8]

print(f"  total_acc_x: mean={uci_lay_acc_x.mean():+.4f}")
print(f"  total_acc_y: mean={uci_lay_acc_y.mean():+.4f}")
print(f"  total_acc_z: mean={uci_lay_acc_z.mean():+.4f}")
uci_lay_gravity = "X" if abs(uci_lay_acc_x.mean()) > 0.8 else ("Y" if abs(uci_lay_acc_y.mean()) > 0.8 else "Z")
print(f"\n  🎯 GRAVITY detected on: {uci_lay_gravity}-axis")

print("\n" + "=" * 60)
print("🔍 ANALYSIS:")
print("=" * 60)
print(f"  Your phone gravity axis:     {gravity_axis}")
print(f"  UCI STANDING gravity axis:   {uci_gravity_axis}")
print(f"  UCI LAYING gravity axis:     {uci_lay_gravity}")

if gravity_axis == uci_lay_gravity and gravity_axis != uci_gravity_axis:
    print(f"\n  ⚠️  YOUR DATA MATCHES UCI LAYING ORIENTATION!")
    print(f"      This explains why the model predicts LAYING.")
    print(f"\n  💡 SOLUTION: Rotate/remap your axes to match UCI STANDING orientation.")
    print(f"      UCI STANDING has gravity on {uci_gravity_axis}-axis")
    print(f"      Your data has gravity on {gravity_axis}-axis")
elif gravity_axis != uci_gravity_axis:
    print(f"\n  ⚠️  AXIS MISMATCH DETECTED!")
    print(f"      Need to remap axes to match UCI orientation.")

COMPARING YOUR DATA vs UCI HAR DATA

📊 YOUR STANDING DATA (first 128 samples after downsampling):
------------------------------------------------------------
  acc_x:  mean=+1.0008, std=0.0093, range=[+0.971, +1.032]
  acc_y:  mean=+0.0130, std=0.0074, range=[-0.006, +0.034]
  acc_z:  mean=-0.0185, std=0.0256, range=[-0.060, +0.042]
  gyro_x: mean=+0.0155, std=0.0546
  gyro_y: mean=-0.0131, std=0.0546
  gyro_z: mean=-0.0047, std=0.0257

  🎯 GRAVITY detected on: X-axis (value ≈ 1.00g)

📊 UCI HAR STANDING DATA (sample #0)
------------------------------------------------------------
  total_acc_x: mean=+0.9938, std=0.0195, range=[+0.928, +1.055]
  total_acc_y: mean=-0.2675, std=0.0098, range=[-0.293, -0.239]
  total_acc_z: mean=+0.1387, std=0.0199, range=[+0.024, +0.173]
  gyro_x:      mean=+0.1523, std=0.1031
  gyro_y:      mean=-0.0079, std=0.1422
  gyro_z:      mean=+0.0457, std=0.0307

  🎯 GRAVITY detected on: X-axis (value ≈ 0.99g)

📊 UCI HAR LAYING DATA (for comparison)
-----------